In [9]:
class Access(object):

    def __getattr__(self, name):
        '''
        该方法定义了你试图访问一个不存在的属性时的行为。
        因此，重载该方法可以实现捕获错误拼写然后进行重定向, 
        或者对一些废弃的属性进行警告。
        '''
        print('__getattr__')
        return super(Access, self).__getattr__(name)

    def __setattr__(self, name, value):
        ''' 
        是实现封装的解决方案，它定义了你对属性进行赋值和修改操作时的行为。
        不管对象的某个属性是否存在,它都允许你为该属性进行赋值,因此你可以为属性的值进行自定义操作。
        有一点需要注意，实现_setattr_时要避免"无限递归"的错误，下面的代码示例中会提到。
        '''
        print('__setattr__')
        return super(Access, self).__setattr__(name, value)

    def __delattr__(self, name):
        print('__delattr__')
        return super(Access, self).__delattr__(name)

    def __getattribute__(self, name):
        #定义了你的属性被访问时的行为
        print('__getattribute__')
        return super(Access, self).__getattribute__(name)

access = Access()
access.attr1 = True  # __setattr__调用
#access.attr1  # 属性存在,只有__getattribute__调用
print(access.attr1)
try:
    access.attr2  # 属性不存在, 先调用__getattribute__, 后调用__getattr__
except AttributeError:
    pass
del access.attr1  # __delattr__调用

__setattr__
__getattribute__
True
__getattribute__
__getattr__
__delattr__


In [10]:
class Access2(object):
    a = []

ac = Access2()
ac.b = []
print(ac.b)
print(ac.c)

[]


AttributeError: 'Access2' object has no attribute 'c'

In [31]:
a = [1,2,3]
b_iter = iter(a)

In [43]:
a.reverse()
b = reversed(a)

In [38]:
try:
    print(next(b_iter))  # 错误用法
except StopIteration:
    print('end')

end


In [53]:
from copy import *

In [54]:
deepcopy(a)

[1, 2, 3]

In [6]:
class Meter(object):
    '''Descriptor for a meter.'''
    def __init__(self, value=0.0):
        self.value = float(value)
    def __get__(self, instance, owner):
        return self.value
    def __set__(self, instance, value):
        self.value = float(value)
 
class Foot(object):
    '''Descriptor for a foot.'''
    def __get__(self, instance, owner):
        return instance.meter * 3.2808
    def __set__(self, instance, value):
        instance.meter = float(value) / 3.2808

class Distance(object):
    meter = Meter()
    foot = Foot()

d = Distance()
print(d.meter, d.foot ) # 0.0, 0.0
d.meter = 1
print(d.meter, d.foot ) # 1.0 3.2808
d.meter = 2
print(d.meter, d.foot)  # 2.0 6.5616

0.0 0.0
1.0 3.2808
2.0 6.5616


In [17]:
from collections import *
from copy import *

In [10]:
a = UserDict([('1', '2')])
a

{'1': '2'}

In [18]:
from typing import List
 
def greet_users(names: List[str]) -> None:
    for name in names:
        a = copy(name)
        print(f"Hello, {a}!")
 
# 使用函数
greet_users(["Alice", "Bob", "Charlie"])

Hello, Alice!
Hello, Bob!
Hello, Charlie!


In [19]:
dict.fromkeys(["Alice", "Bob", "Charlie"])

{'Alice': None, 'Bob': None, 'Charlie': None}

In [20]:
import sys
from unicodedata import name

print(sys.version)
print()
print('sys.stdout.isatty():', sys.stdout.isatty())
print('sys.stdout.encoding:', sys.stdout.encoding)
print()

test_chars = [
    '\N{HORIZONTAL ELLIPSIS}',       # exists in cp1252, not in cp437
    '\N{INFINITY}',                  # exists in cp437, not in cp1252
    '\N{CIRCLED NUMBER FORTY TWO}',  # not in cp437 or in cp1252
]

for char in test_chars:
    print(f'Trying to output {name(char)}:')
    print(char)


3.10.10 (tags/v3.10.10:aad5f6a, Feb  7 2023, 17:20:36) [MSC v.1929 64 bit (AMD64)]

sys.stdout.isatty(): False
sys.stdout.encoding: UTF-8

Trying to output HORIZONTAL ELLIPSIS:
…
Trying to output INFINITY:
∞
Trying to output CIRCLED NUMBER FORTY TWO:
㊷


In [26]:
'\N{CIRCLED NUMBER FORTY TWO}'

'㊷'

In [27]:
'A'.casefold()

'a'

In [35]:
from collections import *
c = namedtuple('c', 'a b c'.split(' '), defaults=[1,2])
c(0)


c(a=0, b=1, c=2)

In [32]:
c._field_defaults

{'b': 1, 'c': 2}

In [40]:
from dataclasses import dataclass, field

@dataclass
class Product:
    name: str
    price: float = field(default=0.0)
    tags: list = field(default_factory=list)
    description: str = field(default="", repr=False)
    discount: float = field(default=0.0, init=False, compare=False)

# 创建 Product 实例
product1 = Product(name="Laptop")
product2 = Product(name="Smartphone", price=999.99, description="Latest model")

# 修改和打印属性
product1.tags.append("electronics")
product2.tags.append("mobile")

product1.discount = 0.1

print(product1)  # 输出: Product(name='Laptop', price=0.0, tags=['electronics'])
print(product2)  # 输出: Product(name='Smartphone', price=999.99, tags=['mobile'])

print(product1.description)  # 输出: ""
print(product2.description)  # 输出: "Latest model"

print(product1.discount)  # 输出: ""
print(product2.discount)  # 输出: "Latest model"

Product(name='Laptop', price=0.0, tags=['electronics'], discount=0.1)
Product(name='Smartphone', price=999.99, tags=['mobile'], discount=0.0)

Latest model
0.1
0.0


In [65]:
from typing import *
from dataclasses import *
@dataclass
class ClubMember:
    name: str
    guests: list = field(default_factory=list)

@dataclass
class HackerClubMember(ClubMember):                         # <1>
    all_handles: ClassVar[set[str]] = set()                # <2>
    handle: str = ''                                        # <3>

    def __post_init__(self):
        cls = self.__class__                                # <4>
        if self.handle == '':                               # <5>
            self.handle = self.name.split()[0]
        if self.handle in cls.all_handles:                  # <6>
            msg = f'handle {self.handle!r} already exists.'
            raise ValueError(msg)
        cls.all_handles.add(self.handle)                    # <7>

@dataclass
class HackerClubMember2(ClubMember):                         # <1>
    all_handles = set()              # <2>
    handle: str = ''                                        # <3>

    def __post_init__(self):
        cls = self.__class__                                # <4>
        if self.handle == '':                               # <5>
            self.handle = self.name.split()[0]
        if self.handle in cls.all_handles:                  # <6>
            msg = f'handle {self.handle!r} already exists.'
            raise ValueError(msg)
        cls.all_handles.add(self.handle)                    # <7>

In [62]:
anna = HackerClubMember2('Anna Ravenscroft', handle='AnnaRaven')
HackerClubMember2.all_handles

{'AnnaRaven'}

In [81]:
@dataclass
class MyClass:
    x: int
    y: int
    factor: InitVar[int]  # 仅用于初始化，不会成为实例属性
    product: int = field(init=False)  # 不在初始化参数中，由其他逻辑设置

    def __post_init__(self, factor):
        self.product = self.x * self.y * factor

    def __post_init__(self, factor):
        self.product = self.x * self.y * factor * 2
    
    def __post_init__(self, factor):
        self.product = self.x * self.y * factor * 4

obj = MyClass(x=2, y=3, factor=4)
print(obj)
#print(obj.product)  # 输出 24 (2 * 3 * 4)
print(hasattr(obj, 'factor'))  # 输出 False，说明 factor 不是实例属性


MyClass(x=2, y=3, product=96)
False


In [2]:
l1 = [1,[2]]
l2 = l1[:]
l1[1].append(2)
l2

[1, [2, 2]]

In [11]:
from typing import *
from collections import *
from decimal import *
from abc import *

class C(NamedTuple):
    name : str
    fid : int

Customer = C('A', 1)
Customer

C(name='A', fid=1)

In [13]:
Optional[str]

typing.Optional[str]

In [18]:
a = 0.1111
f'{a:.2%}'

'11.11%'

In [124]:
class ExampleClass:
    def example_method(self):
        a = 10
        b = 20
        c = a + b
        print("Locals inside the method:", locals())
        #print(type(self).__name__)
    
    def __call__(self): #把类变成一个函数
        print(type(self).__name__)


obj = ExampleClass()
obj.example_method()
obj()

Locals inside the method: {'self': <__main__.ExampleClass object at 0x0000026D1D433340>, 'a': 10, 'b': 20, 'c': 30}
ExampleClass


In [105]:
print(obj.a)
print(obj.a)
print(obj.a)

10
10
10


In [107]:
class Counter:
    def __init__(self):
        self.count = 0
 
    def __call__(self):
        self.count += 1
        return self.count
 
# 创建一个计数器实例
counter = Counter()
 
# 调用计数器实例来增加计数
print(counter())  # 输出结果：1
print(counter())  # 输出结果：2
print(counter())  # 输出结果：3

1
2
3


In [131]:
7 ^ 2 ^ 5 ^ 6

6

In [146]:
from functools import *
import operator
reduce(operator.mul, [1,2])

2

In [156]:
from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):  # <1>
        self._cards[position] = value

    def __delitem__(self, position):  # <2>
        del self._cards[position]

    def insert(self, position, value):  # <3>
        self._cards.insert(position, value)


In [163]:
deck = FrenchDeck2()
card = Card(1,2)
len(deck), [x for x in deck][:2]

(52, [Card(rank='2', suit='spades'), Card(rank='3', suit='spades')])

In [170]:
callable(deck)

False

In [172]:
a = []
a.extend((1,2,3))
a

[1, 2, 3]

[2, 1, 3]

In [183]:
import random
rand = random.SystemRandom()
rand.shuffle(a)

while True:
    try:
        print(a.pop())
    except IndexError:
        raise LookupError('pick from empty a')

LookupError: pick from empty a

In [186]:
def pick(self):  # <5>
    try:
        return self.pop()
    except IndexError:
        pass

In [189]:
# import random
# rand = random.SystemRandom()
# rand.shuffle(a)

# while True:
#     pick(a)